## Junction Risk + Porfolio

In [1]:
import dash
#import dash_core_components as dcc
#import dash_html_components as html
import dash.dcc as dcc
import dash.html as html
from dash.dependencies import Input,Output,State
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import dash_daq as daq
from pickle import load
import cvxopt as opt
from cvxopt import blas, solvers
import sklearn

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage,risk_matrix

from pypfopt import plotting
import copy

ModuleNotFoundError: No module named 'dash'

In [9]:
assets = pd.read_csv('../assets.csv',index_col=0)
filename = 'risk_model_2.p'
loaded_model = load(open(filename, 'rb'))

In [66]:
assets.columns

Index(['ITOT', 'IVV', 'QQQ', 'VTI', 'IJR', 'VPL', 'VWO', 'SUSA', 'HYEM',
       'MGV'],
      dtype='object')

In [10]:
edad = int(input("Edad:"))

Edad: 30


In [11]:
nivel_educativo = int(input("Seleccione su nivel educativo:\n0- Sin educación\n1- Ed. diferencial\n2- Ed. Básica\n3- Ed. Media\n4- CFT o IP\n5- Universitaria\n6- Postgrado\n"))

Seleccione su nivel educativo:
0- Sin educación
1- Ed. diferencial
2- Ed. Básica
3- Ed. Media
4- CFT o IP
5- Universitaria
6- Postgrado
 6


In [5]:
nivel_educativo

5

In [12]:
estado_civil = int(input("Seleccione su estado civil:\n1- Casado\n2- Conviviente\n3- Anulado\n4- Separado\n5 Viudo\n6- Soltero\n7- Divorciado\n"))

Seleccione su estado civil:
1- Casado
2- Conviviente
3- Anulado
4- Separado
5 Viudo
6- Soltero
7- Divorciado
 6


In [13]:
ocup = int(input("Seleccione su estado de ocupación actual:\n0- Desocupado\n1- Ocupado\n"))

Seleccione su estado de ocupación actual:
0- Desocupado
1- Ocupado
 1


In [29]:
tr_numh = int(input("Seleccione el número de habitantes en su hogar:\n1- 1 o 2 personas\n2- 3 o 4 personas\n3- 5 o 6 personas\n5 más de 6 personas\n"))


act_tot = int(input("Ingrese un estimado de tu total de activos en pesos chilenos:\n"))
#estrato = int(input("Ingrese su estrato:\n1- \n6-\n9-"))
#rci_dt  = float(input("Ingrese rci ( entre 0 y 1):\n"))

Seleccione el número de habitantes en su hogar:
1- 1 o 2 personas
2- 3 o 4 personas
3- 5 o 6 personas
5 más de 6 personas
 5
Ingrese un estimado de tu total de activos en pesos chilenos:
 1000000


In [30]:
X_imput = {"edad_ent" : edad,"neduc_ent" : nivel_educativo, "est_civil_ent": estado_civil, "ocup_ent":ocup, "tr_numh": tr_numh,"act_toth":act_tot}
#X_imput = {"edad_ent" : edad,"neduc_ent" : nivel_educativo, "est_civil_ent": estado_civil, "ocup_ent":ocup, "estrato":estrato, "tr_numh": tr_numh,"act_toth":act_tot,"rci_dt":rci_dt}

In [31]:
X_imput = pd.DataFrame(np.array(list(X_imput.values())).reshape(1,-1),columns = list(X_imput.keys()))

In [32]:
risk = loaded_model.predict(X_imput)

In [63]:
risk = [0.80]

In [64]:
def get_asset_allocation(risk,stock_ticker = assets.columns,initial_inv = 100):
    #ipdb.set_trace()
    assets_selected = assets.loc[:,stock_ticker]
    #return_vec = np.array(assets_selected.pct_change().dropna(axis=0)).T
    mu = mean_historical_return(assets,compounding = False)
    #S = CovarianceShrinkage(assets).ledoit_wolf()
    S = risk_matrix(assets)#.sample_cov()
    ef = EfficientFrontier(mu, S)

    #risk = np.sqrt(risk)
    #print(risk[0])
    weights = ef.efficient_risk(risk[0])
    ef.portfolio_performance(verbose=True)

    returns_final= np.dot(assets.loc[:, weights.keys()], np.array(list(weights.values())))
    #returns_sum = np.cumsum(returns_final)
    #print(returns_sum)
    returns_sum_pd = pd.DataFrame(returns_final, index = assets.index )
    returns_sum_pd = returns_sum_pd - returns_sum_pd.iloc[0,:] + initial_inv   
    return weights,returns_sum_pd

In [65]:
w,r = get_asset_allocation(risk)
print(w)

Expected annual return: 17.2%
Annual volatility: 21.1%
Sharpe Ratio: 0.72
OrderedDict([('ITOT', 5.869162e-10), ('IVV', 6.108616e-10), ('QQQ', 0.999999996196945), ('VTI', 5.775537e-10), ('IJR', 5.29845e-10), ('VPL', 1.472618e-10), ('VWO', 1.223078e-10), ('SUSA', 5.993193e-10), ('HYEM', 9.73862e-11), ('MGV', 5.31667e-10)])


## 2d0

Mappear el riesgo del ML correctamente al riesgo de los porftolios.
* El riesgo de los porfolios está bounded
* Poner varios assets 
* acTuaizar los assets
